# BPML - Final Submission
Dataset yang digunakan merupakan data penyakit tanaman tomat yang terdiri dari 8 kelas.

### Menentukan ukuran gambar

In [2]:
height = 128
width = 128
depth = 3

### menetukan directory data

In [3]:
import os
base_dir = 'tomato-desease'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
os.listdir(train_dir)

['Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Tomato___Target_Spot',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus']

### Membuat image data generator

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_datagen = ImageDataGenerator(
    rotation_range=25, 
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.2, 
    zoom_range=0.2,
    horizontal_flip = True,
    rescale = 1./255,
    fill_mode = 'nearest'
)

In [ ]:
### Load data

In [5]:
print('Data Training')
train_data = image_datagen.flow_from_directory(
    train_dir,
    target_size = (height,width),
    batch_size = 10,
    class_mode = 'binary'
)

print('Data Testing')
test_data = image_datagen.flow_from_directory(
    test_dir,
    target_size = (height,width),
    batch_size = 10,
    class_mode = 'binary'
)

Data Training
Found 10912 images belonging to 8 classes.
Data Testing
Found 2666 images belonging to 8 classes.


In [ ]:
### Arsitektur Model

In [13]:
# from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

# def restnet():
#     model_restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(256,256,3))
#     output = model_restnet.layers[-1].output
#     output = keras.layers.Flatten()(output)
#     model_restnet = Model(model_restnet.input, output=output)
#     return model_restnet

def cnn(input_shape):
    name = 'cnn'
    model = Sequential(name=name)
    model.add(Conv2D(16, (3,3), activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))
    model.add(Conv2D(16, (3,3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(16, (3,3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))
    model.add(Conv2D(16, (5,5), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(16, (5,5), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(8, activation='sigmoid'))
    return model

In [14]:
input_shape = (height, width, depth)
model = cnn(input_shape)
model.summary()

Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 126, 126, 16)      448       
_________________________________________________________________
batch_normalization_5 (Batch (None, 126, 126, 16)      64        
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 63, 63, 16)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 63, 63, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 61, 61, 16)        2320      
_________________________________________________________________
batch_normalization_6 (Batch (None, 61, 61, 16)        64        
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 59, 59, 16)        2320    

In [ ]:
### Compile Model

In [8]:
from tensorflow.keras.optimizers import Adam

model.compile(
    loss= 'sparse_categorical_crossentropy', 
    optimizer= Adam(lr=1.000e-3, decay=1.000e-3 / 2),
    metrics= ['accuracy']
)

In [ ]:
### Fitting model

In [10]:
num_epochs = 10
history = model.fit(
    train_data,
    steps_per_epoch=100,
    epochs=num_epochs,
    validation_data=test_data,
    validation_steps=10,
    verbose=1
)

Train for 100 steps, validate for 10 steps
Epoch 1/10
100/100 [==============================] - 49s 489ms/step - loss: 1.5189 - accuracy: 0.4340 - val_loss: 3.1697 - val_accuracy: 0.1500
Epoch 2/10
100/100 [==============================] - 48s 483ms/step - loss: 1.4325 - accuracy: 0.4760 - val_loss: 3.6746 - val_accuracy: 0.1900
Epoch 3/10
100/100 [==============================] - 51s 509ms/step - loss: 1.3933 - accuracy: 0.4980 - val_loss: 1.8195 - val_accuracy: 0.3500
Epoch 4/10
100/100 [==============================] - 49s 488ms/step - loss: 1.2849 - accuracy: 0.5570 - val_loss: 1.6643 - val_accuracy: 0.4600
Epoch 5/10
100/100 [==============================] - 56s 559ms/step - loss: 1.2376 - accuracy: 0.5620 - val_loss: 1.1516 - val_accuracy: 0.5500
Epoch 6/10
100/100 [==============================] - 48s 479ms/step - loss: 1.2428 - accuracy: 0.5750 - val_loss: 1.6472 - val_accuracy: 0.4600
Epoch 7/10
100/100 [==============================] - 46s 456ms/step - loss: 1.2087 - a

In [ ]:
### Plot perubahan akurasi

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1, len(acc) + 1)

#Train and validation accuracy
plt.plot(epochs, acc, 'blue', label='Training accurarcy')
plt.plot(epochs, val_acc, 'black', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()
plt.show()

In [ ]:
### Menyimpan model dalam format .tflite

In [ ]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with tf.io.gfile('model.tflite', 'wb') as f:
    f.write(tflite_model)